In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import ast
from sklearn.feature_extraction import DictVectorizer, FeatureHasher
from itertools import compress


In [2]:
def dict_to_onehot(attribute_df, attribute_name):
    """
    
    """
    result_df = pd.DataFrame()
    for i in range(attribute_df.shape[0]):
        if i % 5000 == 0: print('Line ' + str(i) + ' reached.')
        all_atts = ast.literal_eval(attribute_df.loc[i, 'attributes'])
        bus_id = attribute_df.loc[i, 'business_id']
        if attribute_name in all_atts.keys():
            data = ast.literal_eval(all_atts[attribute_name])
            row = pd.DataFrame.from_dict({i: data}, orient = 'index').astype(int)
            row['business_id'] = bus_id
            result_df = result_df.append(row)
    result_df = result_df.add_prefix(attribute_name + '_')
    result_df['business_id'] = result_df[attribute_name + '_business_id']
    result_df.drop(attribute_name + '_business_id', axis = 1)
    return(result_df)

In [2]:
reviews = pd.read_csv('restaurant_reviews.csv')

In [3]:
businesses = pd.read_csv('businesses_df.csv')

In [76]:
# X = reviews_head['attributes']
# X = reviews['attributes']
bus_atts = []
bus_atts = businesses.loc[:,['attributes', 'business_id']]

In [13]:
bus_atts.loc[0,:]

attributes     {'BikeParking': 'False', 'BusinessAcceptsCredi...
business_id                               Apn5Q_b6Nz61Tq4XzPdf9A
Name: 0, dtype: object

In [77]:
bad_indices = []
i = 0
for n in bus_atts.loc[:,'attributes']:
    if isinstance(n, float): 
        bad_indices.append(i)
    i+=1

In [78]:
print(bad_indices[0:10])
len(bad_indices)

[9, 94, 133, 284, 418, 479, 495, 743, 867, 893]


753

In [79]:
businesses.drop(bad_indices, axis = 0, inplace = True)
bus_atts.drop(bad_indices, axis = 0, inplace = True)

In [20]:
businesses.iloc[9,:]

Unnamed: 0                                                     22
address                                           4131 N 83rd Ave
attributes      {'Ambience': "{'romantic': False, 'intimate': ...
business_id                                3ByGQOVgds2YEu6kzl-XEQ
categories      Nightlife, Bars, American (Traditional), Sport...
city                                                      Phoenix
hours                                                         NaN
is_open                                                         1
latitude                                                  33.4953
longitude                                                -112.236
name                                                     Rib Shop
neighborhood                                                  NaN
postal_code                                                 85033
review_count                                                    3
stars                                                         2.5
state     

In [7]:
X = reviews['attributes']

In [80]:
np.sum(isinstance(n, float) for n in bus_atts.loc[:, 'attributes'])

0

In [81]:
businesses.reset_index(drop = True, inplace = True)
bus_atts.reset_index(drop = True, inplace = True)

In [82]:
print(bus_atts.shape)
print(len(businesses))

(34626, 2)
34626


In [83]:
bus_atts.head()

,attributes,business_id
0,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A
1,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",O8S5hYJ1SMc8fA4QBtVujA
2,"{'Alcohol': 'none', 'BusinessAcceptsCreditCard...",6OuOZAok8ikONMS_T3EzXg
3,"{'Alcohol': 'full_bar', 'Ambience': ""{'romanti...",tZnSodhPwNr4bzrwJ1CSbw
4,"{'Alcohol': 'full_bar', 'Ambience': ""{'romanti...",_c3ixq9jYKxhLUB0czi0ug


In [37]:
atts_to_fix = ['Ambience', 'BusinessParking', 'GoodForMeal', 'BestNights', 'Music']
all_atts = pd.DataFrame()
for i in range(5):
    if i % 5000 == 0 : print( str(i))
    atts_dict = ast.literal_eval(bus_atts.loc[i,'attributes'])
    bus_id = bus_atts.loc[i, 'business_id']
    att_df = pd.DataFrame({'attributes': atts_dict, 'business_id':bus_id})
    print(att_df)
    all_atts.append(att_df)

0
                                                                   attributes  \
BikeParking                                                             False   
BusinessAcceptsCreditCards                                               True   
BusinessParking             {'garage': False, 'street': True, 'validated':...   
GoodForKids                                                              True   
HasTV                                                                    True   
NoiseLevel                                                            average   
OutdoorSeating                                                          False   
RestaurantsAttire                                                      casual   
RestaurantsDelivery                                                     False   
RestaurantsGoodForGroups                                                 True   
RestaurantsPriceRange2                                                      2   
RestaurantsReservations   

In [51]:
'Ambience' in all_atts[0].keys()

False

In [47]:
t = ast.literal_eval(bus_atts.loc[1,'attributes'])
am = ast.literal_eval(t['Ambience'])

tm = pd.DataFrame.from_dict({1:am}, orient = 'index')
tm['business_id'] = bus_atts.loc[1, 'business_id']
tm

,romantic,intimate,classy,hipster,touristy,trendy,upscale,casual,business_id
1,False,False,False,False,False,False,False,False,O8S5hYJ1SMc8fA4QBtVujA


In [52]:
bus_atts.shape[0]

34626

In [131]:
ambience_df = dict_to_onehot(bus_atts, 'Ambience')

Line 0 reached.
Line 5000 reached.
Line 10000 reached.
Line 15000 reached.
Line 20000 reached.
Line 25000 reached.
Line 30000 reached.


In [132]:
ambience_df.head()

,Ambience_business_id,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,business_id
1,O8S5hYJ1SMc8fA4QBtVujA,0,0,NaN,0,0,0,0,0,0,O8S5hYJ1SMc8fA4QBtVujA
3,tZnSodhPwNr4bzrwJ1CSbw,0,0,NaN,0,0,0,0,0,0,tZnSodhPwNr4bzrwJ1CSbw
4,_c3ixq9jYKxhLUB0czi0ug,1,0,0.0,0,0,0,0,0,0,_c3ixq9jYKxhLUB0czi0ug
5,bBUMib8l6Me1ZB1_Qkezkg,0,0,NaN,0,0,0,0,0,0,bBUMib8l6Me1ZB1_Qkezkg
9,3ByGQOVgds2YEu6kzl-XEQ,0,0,0.0,0,0,0,0,0,0,3ByGQOVgds2YEu6kzl-XEQ


In [133]:
business_parking_df = dict_to_onehot(bus_atts, 'BusinessParking')

Line 0 reached.
Line 5000 reached.
Line 10000 reached.
Line 15000 reached.
Line 20000 reached.
Line 25000 reached.
Line 30000 reached.


In [134]:
business_parking_df.head()

,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,BusinessParking_business_id,business_id
0,0,1,0,0,0,Apn5Q_b6Nz61Tq4XzPdf9A,Apn5Q_b6Nz61Tq4XzPdf9A
1,0,0,0,0,0,O8S5hYJ1SMc8fA4QBtVujA,O8S5hYJ1SMc8fA4QBtVujA
2,0,0,0,0,0,6OuOZAok8ikONMS_T3EzXg,6OuOZAok8ikONMS_T3EzXg
3,0,1,0,0,0,tZnSodhPwNr4bzrwJ1CSbw,tZnSodhPwNr4bzrwJ1CSbw
4,0,0,0,1,0,_c3ixq9jYKxhLUB0czi0ug,_c3ixq9jYKxhLUB0czi0ug


In [135]:
gfm_df = dict_to_onehot(bus_atts, 'GoodForMeal')

Line 0 reached.
Line 5000 reached.
Line 10000 reached.
Line 15000 reached.
Line 20000 reached.
Line 25000 reached.
Line 30000 reached.


In [136]:
gfm_df.head()

,GoodForMeal_dessert,GoodForMeal_latenight,GoodForMeal_lunch,GoodForMeal_dinner,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_business_id,business_id
1,0,0,0,0,0,0,O8S5hYJ1SMc8fA4QBtVujA,O8S5hYJ1SMc8fA4QBtVujA
3,0,0,0,1,0,0,tZnSodhPwNr4bzrwJ1CSbw,tZnSodhPwNr4bzrwJ1CSbw
4,0,0,1,1,0,0,_c3ixq9jYKxhLUB0czi0ug,_c3ixq9jYKxhLUB0czi0ug
5,0,0,1,0,0,0,bBUMib8l6Me1ZB1_Qkezkg,bBUMib8l6Me1ZB1_Qkezkg
6,0,0,0,0,0,0,IbTDLI02Sq50C7B-EWK1sg,IbTDLI02Sq50C7B-EWK1sg


In [137]:
music_df = dict_to_onehot(bus_atts, 'Music')

Line 0 reached.
Line 5000 reached.
Line 10000 reached.
Line 15000 reached.
Line 20000 reached.
Line 25000 reached.
Line 30000 reached.


In [138]:
music_df.head()

,Music_dj,Music_background_music,Music_no_music,Music_karaoke,Music_live,Music_video,Music_jukebox,Music_business_id,business_id
4,0,0,0,0,0,0,0,_c3ixq9jYKxhLUB0czi0ug,_c3ixq9jYKxhLUB0czi0ug
9,0,0,0,0,0,0,1,3ByGQOVgds2YEu6kzl-XEQ,3ByGQOVgds2YEu6kzl-XEQ
25,0,1,0,0,0,0,0,cuXCQM-9VwpZlSneEY1b3w,cuXCQM-9VwpZlSneEY1b3w
28,0,1,0,0,0,0,0,F73T-N5lRNL2Gcpg0Q4KKw,F73T-N5lRNL2Gcpg0Q4KKw
29,0,0,0,1,0,0,0,SUhBnQdG8MWkmjM6wsnVdw,SUhBnQdG8MWkmjM6wsnVdw


In [139]:
best_nights_df = dict_to_onehot(bus_atts, 'BestNights')

Line 0 reached.
Line 5000 reached.
Line 10000 reached.
Line 15000 reached.
Line 20000 reached.
Line 25000 reached.
Line 30000 reached.


In [140]:
best_nights_df.head()

,BestNights_monday,BestNights_tuesday,BestNights_friday,BestNights_wednesday,BestNights_thursday,BestNights_sunday,BestNights_saturday,BestNights_business_id,business_id
4,0,1,1,0,0,0,1,_c3ixq9jYKxhLUB0czi0ug,_c3ixq9jYKxhLUB0czi0ug
28,0,0,0,0,0,0,0,F73T-N5lRNL2Gcpg0Q4KKw,F73T-N5lRNL2Gcpg0Q4KKw
30,0,0,0,0,0,0,0,pnYDtPaVw_xRBHm9y0y4gg,pnYDtPaVw_xRBHm9y0y4gg
46,0,0,0,0,0,0,1,XjbPr3o-YTsticeavLjTEg,XjbPr3o-YTsticeavLjTEg
55,0,0,1,0,0,1,1,Gv8u5bX_Kd7HG7jkeF_vWA,Gv8u5bX_Kd7HG7jkeF_vWA


In [63]:
amb_bp_df = ambience_df.merge(business_parking_df, 
                              left_on = 'Ambience_business_id', 
                              right_on = 'BusinessParking_business_id', 
                              how = 'outer')

In [64]:
amb_bp_df.head()

,Ambience_business_id,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,Ambience_upscale,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,BusinessParking_business_id
0,O8S5hYJ1SMc8fA4QBtVujA,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,O8S5hYJ1SMc8fA4QBtVujA
1,tZnSodhPwNr4bzrwJ1CSbw,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,tZnSodhPwNr4bzrwJ1CSbw
2,_c3ixq9jYKxhLUB0czi0ug,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,_c3ixq9jYKxhLUB0czi0ug
3,bBUMib8l6Me1ZB1_Qkezkg,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,bBUMib8l6Me1ZB1_Qkezkg
4,3ByGQOVgds2YEu6kzl-XEQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3ByGQOVgds2YEu6kzl-XEQ


In [65]:
all_atts_df = amb_bp_df.merge(gfm_df, 
                              left_on = ['Ambience_business_id', 'BusinessParking_business_id'],
                              right_on = ['GoodForMeal_business_id', 'GoodForMeal_business_id'],
                              how = 'outer')

In [69]:
all_atts_df.shape

(36080, 23)

In [70]:
all_atts_df.to_csv('ambience_parking_gfm.csv')

In [98]:
all_atts_df = pd.read_csv('ambience_parking_gfm.csv')

/hpc/applications/anaconda/3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [99]:
all_atts_df.head()

,Unnamed: 0,Ambience_business_id,Ambience_casual,Ambience_classy,Ambience_divey,Ambience_hipster,Ambience_intimate,Ambience_romantic,Ambience_touristy,Ambience_trendy,...,BusinessParking_lot,BusinessParking_valet,BusinessParking_business_id,GoodForMeal_dessert,GoodForMeal_latenight,GoodForMeal_lunch,GoodForMeal_dinner,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_business_id
0,0,O8S5hYJ1SMc8fA4QBtVujA,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,O8S5hYJ1SMc8fA4QBtVujA,0.0,0.0,0.0,0.0,0.0,0.0,O8S5hYJ1SMc8fA4QBtVujA
1,1,tZnSodhPwNr4bzrwJ1CSbw,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,tZnSodhPwNr4bzrwJ1CSbw,0.0,0.0,0.0,1.0,0.0,0.0,tZnSodhPwNr4bzrwJ1CSbw
2,2,_c3ixq9jYKxhLUB0czi0ug,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,_c3ixq9jYKxhLUB0czi0ug,0.0,0.0,1.0,1.0,0.0,0.0,_c3ixq9jYKxhLUB0czi0ug
3,3,bBUMib8l6Me1ZB1_Qkezkg,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,bBUMib8l6Me1ZB1_Qkezkg,0.0,0.0,1.0,0.0,0.0,0.0,bBUMib8l6Me1ZB1_Qkezkg
4,4,3ByGQOVgds2YEu6kzl-XEQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3ByGQOVgds2YEu6kzl-XEQ,0.0,0.0,0.0,0.0,0.0,0.0,3ByGQOVgds2YEu6kzl-XEQ


In [108]:
ambience_def = list(isinstance(n, str) for n in all_atts_df.loc[:, 'Ambience_business_id'])
bp_def = list(isinstance(n, str) for n in all_atts_df.loc[:, 'BusinessParking_business_id'])
gfm_def = list(isinstance(n, str) for n in all_atts_df.loc[:, 'GoodForMeal_business_id'])
for i in range(all_atts_df.shape[0]):
    if i% 1000 == 0: print(i)
    if ambience_def[i]:
        all_atts_df.loc[i,'business_id'] = all_atts_df.loc[i, 'Ambience_business_id']
    elif bp_def[i]:
        all_atts_df.loc[i,'business_id'] = all_atts_df.loc[i, 'BusinessParking_business_id']
    elif gfm_def[i]:
        all_atts_df.loc[i,'business_id'] = all_atts_df.loc[i, 'GoodForMeal_business_id']

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000


In [109]:
np.sum(isinstance(n, str) for n in all_atts_df.loc[:, 'business_id'])

36080

In [121]:
all_atts_df.drop_duplicates(inplace = True, subset = 'business_id')

In [122]:
all_atts_df.shape

(32519, 25)

In [126]:
businesses_with_atts = businesses.merge(all_atts_df, on = 'business_id')

In [128]:
businesses_with_atts.to_csv('ambience_parking_gfm.csv')

In [2]:
businesses_with_atts = pd.read_csv('ambience_parking_gfm.csv')

In [3]:
businesses_with_atts.head()

,Unnamed: 0,Unnamed: 0_x,address,attributes,business_id,categories,city,hours,is_open,latitude,...,BusinessParking_lot,BusinessParking_valet,BusinessParking_business_id,GoodForMeal_dessert,GoodForMeal_latenight,GoodForMeal_lunch,GoodForMeal_dinner,GoodForMeal_breakfast,GoodForMeal_brunch,GoodForMeal_business_id
0,0,0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,...,0.0,0.0,Apn5Q_b6Nz61Tq4XzPdf9A,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,1335 rue Beaubien E,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",Montréal,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",0,45.540503,...,0.0,0.0,O8S5hYJ1SMc8fA4QBtVujA,0.0,0.0,0.0,0.0,0.0,0.0,O8S5hYJ1SMc8fA4QBtVujA
2,2,7,"3417 Derry Road E, Unit 103","{'Alcohol': 'none', 'BusinessAcceptsCreditCard...",6OuOZAok8ikONMS_T3EzXg,"Restaurants, Thai",Mississauga,NaN,1,43.712946,...,0.0,0.0,6OuOZAok8ikONMS_T3EzXg,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,11,595 Markham Street,"{'Alcohol': 'full_bar', 'Ambience': ""{'romanti...",tZnSodhPwNr4bzrwJ1CSbw,"Cajun/Creole, Southern, Restaurants",Toronto,"{'Tuesday': '17:0-1:0', 'Wednesday': '17:0-1:0...",0,43.664125,...,0.0,0.0,tZnSodhPwNr4bzrwJ1CSbw,0.0,0.0,0.0,1.0,0.0,0.0,tZnSodhPwNr4bzrwJ1CSbw
4,4,12,2801 N 15th Ave,"{'Alcohol': 'full_bar', 'Ambience': ""{'romanti...",_c3ixq9jYKxhLUB0czi0ug,"Bars, Sports Bars, Dive Bars, Burgers, Nightli...",Phoenix,"{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",1,33.479807,...,1.0,0.0,_c3ixq9jYKxhLUB0czi0ug,0.0,0.0,1.0,1.0,0.0,0.0,_c3ixq9jYKxhLUB0czi0ug


In [56]:
atts = []
all_atts_df = pd.DataFrame()
for i in range(businesses_with_atts.shape[0]):  # 
    if i % 5000 == 0: print( i)
    atts = []
    non_dict_atts = {}
    buss_id = businesses_with_atts.loc[i, 'business_id']
    attributes = businesses_with_atts.loc[i, 'attributes']
    att_dict = ast.literal_eval(attributes)
    for att, val in att_dict.items():
        if not (isinstance(val, dict)):
            atts.append(att)
    for key in atts:
        non_dict_atts[key] = att_dict[key]
    atts_df = pd.DataFrame.from_dict({i:non_dict_atts}, orient = 'index')
    atts_df['business_id'] = buss_id
    all_atts_df = all_atts_df.append(atts_df)



0
5000
10000
15000
20000
25000
30000


In [57]:
all_atts_df[:20]

,AcceptsInsurance,AgesAllowed,Alcohol,Ambience,BYOB,BYOBCorkage,BestNights,BikeParking,BusinessAcceptsBitcoin,BusinessAcceptsCreditCards,...,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,business_id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,True,...,False,True,2,True,NaN,True,NaN,NaN,NaN,Apn5Q_b6Nz61Tq4XzPdf9A
1,NaN,NaN,beer_and_wine,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,True,NaN,False,...,False,True,2,True,True,False,NaN,NaN,free,O8S5hYJ1SMc8fA4QBtVujA
2,NaN,NaN,none,NaN,NaN,NaN,NaN,NaN,NaN,True,...,NaN,True,2,NaN,False,True,NaN,NaN,NaN,6OuOZAok8ikONMS_T3EzXg
3,NaN,NaN,full_bar,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,True,NaN,True,...,False,True,2,True,True,True,NaN,False,no,tZnSodhPwNr4bzrwJ1CSbw
4,NaN,NaN,full_bar,"{'romantic': False, 'intimate': False, 'classy...",False,yes_free,"{'monday': False, 'tuesday': True, 'friday': T...",True,NaN,True,...,False,True,1,False,False,True,outdoor,True,no,_c3ixq9jYKxhLUB0czi0ug
5,NaN,NaN,full_bar,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,True,NaN,True,...,True,True,2,True,True,True,NaN,True,no,bBUMib8l6Me1ZB1_Qkezkg
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,...,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,IbTDLI02Sq50C7B-EWK1sg
7,NaN,NaN,full_bar,NaN,NaN,NaN,NaN,True,NaN,True,...,False,True,3,True,NaN,False,NaN,True,free,r89Re4FNgVWHgBfjCVZyVw
8,NaN,NaN,NaN,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,NaN,NaN,True,...,NaN,NaN,1,NaN,True,True,NaN,NaN,free,3ByGQOVgds2YEu6kzl-XEQ
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,...,False,True,1,False,NaN,True,NaN,NaN,no,CeuTRtwsq6w5rztGOyNMPg


In [141]:
all_atts_df.columns

Index(['AcceptsInsurance', 'AgesAllowed', 'Alcohol', 'Ambience', 'BYOB',
       'BYOBCorkage', 'BestNights', 'BikeParking', 'BusinessAcceptsBitcoin',
       'BusinessAcceptsCreditCards', 'BusinessParking', 'ByAppointmentOnly',
       'Caters', 'CoatCheck', 'Corkage', 'DietaryRestrictions', 'DogsAllowed',
       'DriveThru', 'GoodForDancing', 'GoodForKids', 'GoodForMeal',
       'HairSpecializesIn', 'HappyHour', 'HasTV', 'Music', 'NoiseLevel',
       'Open24Hours', 'OutdoorSeating', 'RestaurantsAttire',
       'RestaurantsCounterService', 'RestaurantsDelivery',
       'RestaurantsGoodForGroups', 'RestaurantsPriceRange2',
       'RestaurantsReservations', 'RestaurantsTableService',
       'RestaurantsTakeOut', 'Smoking', 'WheelchairAccessible', 'WiFi',
       'business_id'],
      dtype='object')

In [ ]:
for att in all_atts_df.columns:
    print(all_atts_df.loc[:, att].value_counts())

In [142]:
aad = all_atts_df.drop(['AcceptsInsurance', 'AgesAllowed', 'Ambience', 'BYOB', 'BYOBCorkage', 
                        'BestNights', 'Corkage', 'DietaryRestrictions', 'DogsAllowed',
                        'DriveThru', 'GoodForMeal', 'BusinessParking', 'Open24Hours',
                        'GoodForDancing', 'GoodForKids', 'HairSpecializesIn',
                        'RestaurantsCounterService'], axis = 1)



In [143]:
aad.columns

Index(['Alcohol', 'BikeParking', 'BusinessAcceptsBitcoin',
       'BusinessAcceptsCreditCards', 'ByAppointmentOnly', 'Caters',
       'CoatCheck', 'HappyHour', 'HasTV', 'Music', 'NoiseLevel',
       'OutdoorSeating', 'RestaurantsAttire', 'RestaurantsDelivery',
       'RestaurantsGoodForGroups', 'RestaurantsPriceRange2',
       'RestaurantsReservations', 'RestaurantsTableService',
       'RestaurantsTakeOut', 'Smoking', 'WheelchairAccessible', 'WiFi',
       'business_id'],
      dtype='object')

In [144]:

aad[:20]

,Alcohol,BikeParking,BusinessAcceptsBitcoin,BusinessAcceptsCreditCards,ByAppointmentOnly,Caters,CoatCheck,HappyHour,HasTV,Music,...,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,business_id
0,NaN,False,NaN,True,NaN,NaN,NaN,NaN,True,NaN,...,False,True,2,True,NaN,True,NaN,NaN,NaN,Apn5Q_b6Nz61Tq4XzPdf9A
1,beer_and_wine,True,NaN,False,NaN,False,NaN,NaN,True,NaN,...,False,True,2,True,True,False,NaN,NaN,free,O8S5hYJ1SMc8fA4QBtVujA
2,none,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,2,NaN,False,True,NaN,NaN,NaN,6OuOZAok8ikONMS_T3EzXg
3,full_bar,True,NaN,True,NaN,True,NaN,NaN,False,NaN,...,False,True,2,True,True,True,NaN,False,no,tZnSodhPwNr4bzrwJ1CSbw
4,full_bar,True,NaN,True,NaN,False,False,True,True,"{'dj': False, 'background_music': False, 'no_m...",...,False,True,1,False,False,True,outdoor,True,no,_c3ixq9jYKxhLUB0czi0ug
5,full_bar,True,NaN,True,NaN,NaN,NaN,NaN,False,NaN,...,True,True,2,True,True,True,NaN,True,no,bBUMib8l6Me1ZB1_Qkezkg
6,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,IbTDLI02Sq50C7B-EWK1sg
7,full_bar,True,NaN,True,NaN,False,NaN,True,False,NaN,...,False,True,3,True,NaN,False,NaN,True,free,r89Re4FNgVWHgBfjCVZyVw
8,NaN,NaN,NaN,True,NaN,NaN,False,NaN,True,"{'dj': False, 'background_music': False, 'no_m...",...,NaN,NaN,1,NaN,True,True,NaN,NaN,free,3ByGQOVgds2YEu6kzl-XEQ
9,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,True,NaN,...,False,True,1,False,NaN,True,NaN,NaN,no,CeuTRtwsq6w5rztGOyNMPg


In [149]:
onehot_dfs = [ambience_df, business_parking_df, gfm_df, music_df, best_nights_df,aad]
for df in onehot_dfs:
    businesses = businesses.merge(df, on = 'business_id', how = 'outer')

In [150]:
businesses.columns

Index(['Unnamed: 0', 'address', 'attributes', 'business_id', 'categories',
       'city', 'hours', 'is_open', 'latitude', 'longitude', 'name',
       'neighborhood', 'postal_code', 'review_count', 'stars', 'state',
       'Ambience_business_id', 'Ambience_casual', 'Ambience_classy',
       'Ambience_divey', 'Ambience_hipster', 'Ambience_intimate',
       'Ambience_romantic', 'Ambience_touristy', 'Ambience_trendy',
       'Ambience_upscale', 'BusinessParking_garage', 'BusinessParking_street',
       'BusinessParking_validated', 'BusinessParking_lot',
       'BusinessParking_valet', 'BusinessParking_business_id',
       'GoodForMeal_dessert', 'GoodForMeal_latenight', 'GoodForMeal_lunch',
       'GoodForMeal_dinner', 'GoodForMeal_breakfast', 'GoodForMeal_brunch',
       'GoodForMeal_business_id', 'Music_dj', 'Music_background_music',
       'Music_no_music', 'Music_karaoke', 'Music_live', 'Music_video',
       'Music_jukebox', 'Music_business_id', 'BestNights_monday',
       'BestNights_t

In [4]:
businesses.shape

(35379, 78)

In [152]:
businesses.to_csv('businesses_df.csv')

In [14]:
businesses['categories'].value_counts()

Restaurants, Mexican                                                                                                                                               595
Restaurants, Pizza                                                                                                                                                 583
Mexican, Restaurants                                                                                                                                               565
Pizza, Restaurants                                                                                                                                                 539
Restaurants, Chinese                                                                                                                                               478
Chinese, Restaurants                                                                                                                                               46

In [72]:
b = businesses.loc[:,['categories', 'business_id']][:20]

In [73]:
b

,categories,business_id
0,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Apn5Q_b6Nz61Tq4XzPdf9A
1,"Breakfast & Brunch, Restaurants, French, Sandw...",O8S5hYJ1SMc8fA4QBtVujA
2,"Restaurants, Thai",6OuOZAok8ikONMS_T3EzXg
3,"Cajun/Creole, Southern, Restaurants",tZnSodhPwNr4bzrwJ1CSbw
4,"Bars, Sports Bars, Dive Bars, Burgers, Nightli...",_c3ixq9jYKxhLUB0czi0ug
5,"Restaurants, Pakistani, Indian, Middle Eastern",bBUMib8l6Me1ZB1_Qkezkg
6,"Delis, Restaurants, Sandwiches",IbTDLI02Sq50C7B-EWK1sg
7,"Nightlife, Bars, American (Traditional), Tapas...",r89Re4FNgVWHgBfjCVZyVw
8,"American (Traditional), Food, Bakeries, Restau...",vJIuDBdu01vCA8y1fwR1OQ
9,"Fast Food, Burgers, Restaurants",gJ5xSt6147gkcZ9Es0WxlA


In [4]:
all_cats = set()
for i in businesses['categories']:
    cats = i.split(', ')
    for c in cats: all_cats.add(c)
print(all_cats)

{'Udon', 'Pool & Billiards', 'Souvenir Shops', 'Supernatural Readings', 'Hot Dogs', 'Pop-up Shops', 'Arcades', 'Cooking Classes', 'Pet Services', 'Salad', 'Uzbek', 'Auto Detailing', 'Indoor Playcentre', 'Books', 'French', 'Active Life', 'Empanadas', 'Party Supplies', 'Barbeque', 'Middle Eastern', 'Kitchen & Bath', 'Korean', 'Tapas Bars', 'Venezuelan', 'Clowns', 'British', "Women's Clothing", 'Escape Games', 'Rock Climbing', 'Sicilian', 'Indian', 'Poutineries', 'Contractors', 'Slovakian', 'Reflexology', 'Pediatricians', 'Restaurants', 'Auto Upholstery', 'Tasting Classes', 'Coffee & Tea', 'Bookstores', 'Water Heater Installation/Repair', 'Creperies', 'Cheese Shops', 'Leather Goods', 'Eyelash Service', 'Tattoo', 'Alternative Medicine', 'Falafel', 'Club Crawl', 'Windshield Installation & Repair', 'Wine Tasting Room', 'Sandwiches', 'Home Decor', 'Arts & Entertainment', 'Cultural Center', 'University Housing', 'Golf Equipment Shops', 'Local Fish Stores', 'Cinema', 'Shaved Ice', 'Poke', 'Mala

In [5]:
cats_df = pd.DataFrame()
for cat in all_cats:
    cats_df[cat] = np.nan
for i in range(businesses.shape[0]):
    if i % 1000 == 0: print('Line ' + str(i) + ' reached.')
    df_row = {}
    for col in cats_df.columns:
        if(str(col) in businesses.loc[i,'categories']):
            df_row[col] = True
        else: df_row[col] = False
    df_row['business_id'] = businesses.loc[i,'business_id']
    cats_df = cats_df.append(df_row, ignore_index = True)

Line 0 reached.
Line 1000 reached.
Line 2000 reached.
Line 3000 reached.
Line 4000 reached.
Line 5000 reached.
Line 6000 reached.
Line 7000 reached.
Line 8000 reached.
Line 9000 reached.
Line 10000 reached.
Line 11000 reached.
Line 12000 reached.
Line 13000 reached.
Line 14000 reached.
Line 15000 reached.
Line 16000 reached.
Line 17000 reached.
Line 18000 reached.
Line 19000 reached.
Line 20000 reached.
Line 21000 reached.
Line 22000 reached.
Line 23000 reached.
Line 24000 reached.
Line 25000 reached.
Line 26000 reached.
Line 27000 reached.
Line 28000 reached.
Line 29000 reached.
Line 30000 reached.
Line 31000 reached.
Line 32000 reached.
Line 33000 reached.
Line 34000 reached.
Line 35000 reached.


In [6]:
less_than_fifty_bus = (np.sum(cats_df[cats_df.columns.difference(['business_id'])], axis = 0) < 50)
cats_to_remove = list(less_than_fifty_bus[less_than_fifty_bus].index)

In [7]:
for cat in cats_to_remove:
    all_cats.remove(cat)

In [8]:
cats_df = pd.DataFrame()
for cat in all_cats:
    cats_df[cat] = np.nan
for i in range(businesses.shape[0]):
    if i % 1000 == 0: print('Line ' + str(i) + ' reached.')
    df_row = {}
    for col in cats_df.columns:
        if(str(col) in businesses.loc[i,'categories']):
            df_row[col] = True
        else: df_row[col] = False
    df_row['business_id'] = businesses.loc[i,'business_id']
    cats_df = cats_df.append(df_row, ignore_index = True)

Line 0 reached.
Line 1000 reached.
Line 2000 reached.
Line 3000 reached.
Line 4000 reached.
Line 5000 reached.
Line 6000 reached.
Line 7000 reached.
Line 8000 reached.
Line 9000 reached.
Line 10000 reached.
Line 11000 reached.
Line 12000 reached.
Line 13000 reached.
Line 14000 reached.
Line 15000 reached.
Line 16000 reached.
Line 17000 reached.
Line 18000 reached.
Line 19000 reached.
Line 20000 reached.
Line 21000 reached.
Line 22000 reached.
Line 23000 reached.
Line 24000 reached.
Line 25000 reached.
Line 26000 reached.
Line 27000 reached.
Line 28000 reached.
Line 29000 reached.
Line 30000 reached.
Line 31000 reached.
Line 32000 reached.
Line 33000 reached.
Line 34000 reached.
Line 35000 reached.


In [9]:
cats_df.head()

,Hot Dogs,Salad,French,Active Life,Barbeque,Middle Eastern,Korean,Tapas Bars,British,Indian,...,Vietnamese,African,Seafood,Soup,Thai,Soul Food,Peruvian,Bistros,Irish,business_id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Apn5Q_b6Nz61Tq4XzPdf9A
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,O8S5hYJ1SMc8fA4QBtVujA
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6OuOZAok8ikONMS_T3EzXg
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,tZnSodhPwNr4bzrwJ1CSbw
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,_c3ixq9jYKxhLUB0czi0ug


In [10]:
cats_df.to_csv('category_df.csv')

In [3]:
reviews.columns

Index(['Unnamed: 0', 'Unnamed: 0_x', 'business_id', 'cool', 'date', 'funny',
       'review_id', 'stars_x', 'text', 'useful', 'user_id', 'Unnamed: 0_y',
       'address', 'attributes', 'categories', 'city', 'hours', 'is_open',
       'latitude', 'longitude', 'name', 'neighborhood', 'postal_code',
       'review_count', 'stars_y', 'state'],
      dtype='object')

In [5]:
text_reviews = reviews.loc[:,['business_id','review_id', 'text']]

In [6]:
text_reviews.head()

,business_id,review_id,text
0,iCQpiavjjPzJ5_3gPD5Ebg,x7mDIiDB3jEiPGPHOmDzyw,The pizza was okay. Not the best I've had. I p...
1,iCQpiavjjPzJ5_3gPD5Ebg,VluIpojME0yKOcRKI5L0Iw,"came here on Monday, no line. its good, but no..."
2,iCQpiavjjPzJ5_3gPD5Ebg,nsB9JAeghk0zOaSulSm9Yw,This place is truly a secret! Its so hidden t...
3,pomGBqfbxcqPv14c3XH-ZQ,dDl8zu1vWPdKGihJrwQbpw,I love this place! My fiance And I go here atl...
4,jtQARsP6P-LbkyjbO1qNGg,LZp4UX5zK3e-c5ZGSeo3kA,Terrible. Dry corn bread. Rib tips were all fa...


In [7]:
text_reviews.to_csv('text_reviews.csv')

In [6]:
businesses.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'address', 'attributes', 'business_id',
       'categories', 'city', 'hours', 'is_open', 'latitude', 'longitude',
       'name', 'neighborhood', 'postal_code', 'review_count', 'stars', 'state',
       'Ambience_business_id', 'Ambience_casual', 'Ambience_classy',
       'Ambience_divey', 'Ambience_hipster', 'Ambience_intimate',
       'Ambience_romantic', 'Ambience_touristy', 'Ambience_trendy',
       'Ambience_upscale', 'BusinessParking_garage', 'BusinessParking_street',
       'BusinessParking_validated', 'BusinessParking_lot',
       'BusinessParking_valet', 'BusinessParking_business_id',
       'GoodForMeal_dessert', 'GoodForMeal_latenight', 'GoodForMeal_lunch',
       'GoodForMeal_dinner', 'GoodForMeal_breakfast', 'GoodForMeal_brunch',
       'GoodForMeal_business_id', 'Music_dj', 'Music_background_music',
       'Music_no_music', 'Music_karaoke', 'Music_live', 'Music_video',
       'Music_jukebox', 'Music_business_id', 'BestNights_monday',
    

In [7]:
categories = pd.read_csv("category_df.csv")

In [8]:
categories.columns

Index(['Unnamed: 0', 'Hot Dogs', 'Salad', 'French', 'Active Life', 'Barbeque',
       'Middle Eastern', 'Korean', 'Tapas Bars', 'British',
       ...
       'Vietnamese', 'African', 'Seafood', 'Soup', 'Thai', 'Soul Food',
       'Peruvian', 'Bistros', 'Irish', 'business_id'],
      dtype='object', length=149)

In [10]:
full_df = categories.merge(businesses, on = 'business_id')

In [12]:
full_df.to_csv("Full_restaurant_df.csv")

In [13]:
full_review_df = reviews.merge(full_df, on = 'business_id')

In [20]:
full_review_df.columns

Index(['Unnamed: 0', 'Unnamed: 0_x_x', 'business_id', 'cool', 'date', 'funny',
       'review_id', 'stars_x', 'text', 'useful',
       ...
       'RestaurantsAttire', 'RestaurantsDelivery', 'RestaurantsGoodForGroups',
       'RestaurantsPriceRange2', 'RestaurantsReservations',
       'RestaurantsTableService', 'RestaurantsTakeOut', 'Smoking',
       'WheelchairAccessible', 'WiFi'],
      dtype='object', length=251)

In [21]:
full_review_df.to_csv('Complete_review_df.csv')

## Adding Review # to each review

In [3]:
reviews = pd.read_csv("restaurant_reviews.csv")

In [23]:
def add_review_num(df, primary_col, sec_col):
    assert(primary_col in df)
    assert(sec_col in df)
    sorted_df = df.sort_values([primary_col, sec_col])
    sorted_df["Review_Num"] = np.zeros(df.shape[0])
    
    idx = 0
    review_count = 1
    
    current_id = sorted_df.loc[idx, primary_col]
    while idx < df.shape[0]:
        if current_id != sorted_df.loc[idx, primary_col]: 
            current_id = sorted_df.loc[idx, primary_col]
            review_count = 1
        sorted_df.loc[idx, "Review_Num"] = review_count
        idx += 1
        review_count += 1
    return sorted_df

In [7]:
reviews.head().sort_values(['business_id','date'])

,Unnamed: 0,Unnamed: 0_x,business_id,cool,date,funny,review_id,stars_x,text,useful,...,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars_y,state
0,0,0,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0.0,x7mDIiDB3jEiPGPHOmDzyw,2.0,The pizza was okay. Not the best I've had. I p...,0.0,...,"{'Monday': '11:0-5:0', 'Tuesday': '11:0-4:0', ...",1,36.109837,-115.174212,Secret Pizza,The Strip,89109,4078,4.0,NV
1,1,788,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-10-25,0.0,VluIpojME0yKOcRKI5L0Iw,3.0,"came here on Monday, no line. its good, but no...",0.0,...,"{'Monday': '11:0-5:0', 'Tuesday': '11:0-4:0', ...",1,36.109837,-115.174212,Secret Pizza,The Strip,89109,4078,4.0,NV
2,2,3314,iCQpiavjjPzJ5_3gPD5Ebg,0,2016-05-22,0.0,nsB9JAeghk0zOaSulSm9Yw,5.0,This place is truly a secret! Its so hidden t...,0.0,...,"{'Monday': '11:0-5:0', 'Tuesday': '11:0-4:0', ...",1,36.109837,-115.174212,Secret Pizza,The Strip,89109,4078,4.0,NV
4,4,2,jtQARsP6P-LbkyjbO1qNGg,1,2014-10-23,1.0,LZp4UX5zK3e-c5ZGSeo3kA,1.0,Terrible. Dry corn bread. Rib tips were all fa...,3.0,...,"{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",1,36.241809,-115.234495,H&H BBQ Plus 2,Northwest,89130,115,3.5,NV
3,3,1,pomGBqfbxcqPv14c3XH-ZQ,0,2012-11-13,0.0,dDl8zu1vWPdKGihJrwQbpw,5.0,I love this place! My fiance And I go here atl...,0.0,...,"{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",1,36.298875,-115.280088,Leticia's Mexican Cocina,Centennial,89143,1125,4.0,NV


In [8]:
reviews.head()['business_id']

0    iCQpiavjjPzJ5_3gPD5Ebg
1    iCQpiavjjPzJ5_3gPD5Ebg
2    iCQpiavjjPzJ5_3gPD5Ebg
3    pomGBqfbxcqPv14c3XH-ZQ
4    jtQARsP6P-LbkyjbO1qNGg
Name: business_id, dtype: object

In [ ]:
numbered_reviews = add_review_num(reviews, 'business_id', 'date')

In [ ]:
numbered_reviews.to_csv("Numbered_reviews.csv")

In [ ]:
complete_review_df = pd.read_csv("Complete_review_df.csv")
numbered_reviews.merge(complete_review_df, on = 'review_id')

## One Hot Encoding Neighborhood

In [2]:
reviews = pd.read_csv("Complete_numbered_reviews.csv")

/hpc/applications/anaconda/3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (21,35,216,224,226,227,228,229,230,231,232,233,234,236,238,239,241,242,243,244,245) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# reviews['Review_Num'] = reviews['Review_Num'] + 1

In [13]:
pd.options.display.max_seq_items = 250

In [14]:
reviews.head(50).columns

Index(['business_id_x', 'cool_x', 'date_x', 'funny_x', 'review_id',
       'stars_x_x', 'text_x', 'useful_x', 'user_id_x', 'address', 'attributes',
       'categories', 'city', 'hours', 'is_open', 'latitude', 'longitude',
       'name', 'neighborhood', 'postal_code', 'review_count', 'state',
       'Review_Num', 'address_x', 'attributes_x', 'categories_x', 'city_x',
       'hours_x', 'is_open_x', 'latitude_x', 'longitude_x', 'name_x',
       'neighborhood_x', 'postal_code_x', 'review_count_x', 'state_x',
       'Hot Dogs', 'Salad', 'French', 'Active Life', 'Barbeque',
       'Middle Eastern', 'Korean', 'Tapas Bars', 'British', 'Indian',
       'Poutineries', 'Restaurants', 'Coffee & Tea', 'Creperies', 'Falafel',
       'Sandwiches', 'Arts & Entertainment', 'Poke', 'Noodles',
       'Do-It-Yourself Food', 'Bakeries', 'Ethiopian', 'Asian Fusion',
       'Hawaiian', 'Japanese', 'Local Flavor', 'Vegan', 'Taiwanese', 'Tex-Mex',
       'Food Court', 'Dance Clubs', 'Cajun/Creole', 'Sushi Bars

In [17]:
reviews = reviews.rename(index = str, columns = {
    'business_id_x' : 'business_id',
    'stars_x_x': 'stars', 
    'cool_x': 'cool',
    'date_x': 'date',
    'funny_x': 'funny', 
    'text_x': 'text',
    'useful_x' : 'useful',
    'user_id_x': 'user_id'
})

In [18]:
reviews.columns

Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text',
       'useful', 'user_id', 'address', 'attributes', 'categories', 'city',
       'hours', 'is_open', 'latitude', 'longitude', 'name', 'neighborhood',
       'postal_code', 'review_count', 'state', 'Review_Num', 'address_x',
       'attributes_x', 'categories_x', 'city_x', 'hours_x', 'is_open_x',
       'latitude_x', 'longitude_x', 'name_x', 'neighborhood_x',
       'postal_code_x', 'review_count_x', 'state_x', 'Hot Dogs', 'Salad',
       'French', 'Active Life', 'Barbeque', 'Middle Eastern', 'Korean',
       'Tapas Bars', 'British', 'Indian', 'Poutineries', 'Restaurants',
       'Coffee & Tea', 'Creperies', 'Falafel', 'Sandwiches',
       'Arts & Entertainment', 'Poke', 'Noodles', 'Do-It-Yourself Food',
       'Bakeries', 'Ethiopian', 'Asian Fusion', 'Hawaiian', 'Japanese',
       'Local Flavor', 'Vegan', 'Taiwanese', 'Tex-Mex', 'Food Court',
       'Dance Clubs', 'Cajun/Creole', 'Sushi Bars', 'Desserts', 'Pa

In [11]:
pd.set_option('display.max_columns', 300)

In [20]:
reviews.drop(list(reviews.filter(regex = '_x')), axis = 1, inplace = True)

ValueError: cannot reindex from a duplicate axis

In [9]:
reviews.drop(list(reviews.filter(regex = 'Unnamed:')), axis = 1, inplace = True)

In [10]:
reviews.columns

Index(['business_id_x', 'cool_x', 'date_x', 'funny_x', 'review_id',
       'stars_x_x', 'text_x', 'useful_x', 'user_id_x', 'address',
       ...
       'RestaurantsAttire', 'RestaurantsDelivery', 'RestaurantsGoodForGroups',
       'RestaurantsPriceRange2', 'RestaurantsReservations',
       'RestaurantsTableService', 'RestaurantsTakeOut', 'Smoking',
       'WheelchairAccessible', 'WiFi'],
      dtype='object', length=245)

In [22]:
reviews.drop(['address_x',
       'attributes_x', 'categories_x', 'city_x', 'hours_x', 'is_open_x',
       'latitude_x', 'longitude_x', 'name_x', 'neighborhood_x',
       'postal_code_x', 'review_count_x', 'state_x'], axis = 1, inplace = True)

In [28]:
dummies = pd.get_dummies(reviews.head(100), 
                         prefix = {'state': 'St',
                                   'neighborhood':'nbrhd'}, 
                         columns = ['state', 'neighborhood'])

In [31]:
reviews.to_csv('Complete_numbered_reviews.csv')

In [33]:
reviews.columns

Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text',
       'useful', 'user_id', 'address', 'attributes', 'categories', 'city',
       'hours', 'is_open', 'latitude', 'longitude', 'name', 'neighborhood',
       'postal_code', 'review_count', 'state', 'Review_Num', 'Hot Dogs',
       'Salad', 'French', 'Active Life', 'Barbeque', 'Middle Eastern',
       'Korean', 'Tapas Bars', 'British', 'Indian', 'Poutineries',
       'Restaurants', 'Coffee & Tea', 'Creperies', 'Falafel', 'Sandwiches',
       'Arts & Entertainment', 'Poke', 'Noodles', 'Do-It-Yourself Food',
       'Bakeries', 'Ethiopian', 'Asian Fusion', 'Hawaiian', 'Japanese',
       'Local Flavor', 'Vegan', 'Taiwanese', 'Tex-Mex', 'Food Court',
       'Dance Clubs', 'Cajun/Creole', 'Sushi Bars', 'Desserts', 'Pan Asian',
       'Food Trucks', 'Turkish', 'Beer', 'Venues & Event Spaces',
       'Breakfast & Brunch', 'Bars', 'Pakistani', 'Salvadoran', 'Diners',
       'Italian', 'Tapas/Small Plates', 'Caterers', '

##  Checking Sentiment

In [12]:
sents = pd.read_csv('full_review_sentiments.csv')

In [20]:
sents.shape

(2706326, 5)

In [18]:
sents.head()

,review_id,neg,neu,pos,compound
0,Mc0ecKvOZmDGPDt1UdQcBw,0.171,0.729,0.10,-0.8224
1,Ct9VXpsJtohITDHspduAhQ,0.050,0.770,0.18,0.9593
2,C_pdS4RGgrPrt4ctYOrQ0A,0.029,0.752,0.22,0.8726
3,GSTQtPuRgIGveAAJYaIn3A,0.000,0.740,0.26,0.9370
4,-i9u19L08A4VvhFVfqXK9Q,0.021,0.720,0.26,0.9617


In [17]:
sents.drop('Sentiment', axis = 1, inplace = True)

In [9]:
sents['Sentiment'].map(eval).apply(pd.Series)

,compound,neg,neu,pos
0,-0.8224,0.171,0.729,0.10
1,0.9593,0.050,0.770,0.18
2,0.8726,0.029,0.752,0.22
3,0.9370,0.000,0.740,0.26
4,0.9617,0.021,0.720,0.26


In [21]:
sents.to_csv('full_review_sentiments.csv')

In [23]:
sents.shape

(2706326, 5)

## Checking the df

In [24]:
df = pd.read_csv("sentiment_numbered_reviews.csv")

/hpc/applications/anaconda/3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (22,204,212,214,215,216,217,218,219,220,221,222,224,226,227,229,230,231,232,233) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [52]:
pd.set_option("max_seq_items", 600)

In [36]:
df.columns

Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text',
       'useful', 'user_id', 'address', 'attributes', 'categories', 'city',
       'hours', 'is_open', 'latitude', 'longitude', 'name', 'neighborhood',
       'postal_code', 'review_count', 'state', 'Review_Num', 'Hot Dogs',
       'Salad', 'French', 'Active Life', 'Barbeque', 'Middle Eastern',
       'Korean', 'Tapas Bars', 'British', 'Indian', 'Poutineries',
       'Restaurants', 'Coffee & Tea', 'Creperies', 'Falafel', 'Sandwiches',
       'Arts & Entertainment', 'Poke', 'Noodles', 'Do-It-Yourself Food',
       'Bakeries', 'Ethiopian', 'Asian Fusion', 'Hawaiian', 'Japanese',
       'Local Flavor', 'Vegan', 'Taiwanese', 'Tex-Mex', 'Food Court',
       'Dance Clubs', 'Cajun/Creole', 'Sushi Bars', 'Desserts', 'Pan Asian',
       'Food Trucks', 'Turkish', 'Beer', 'Venues & Event Spaces',
       'Breakfast & Brunch', 'Bars', 'Pakistani', 'Salvadoran', 'Diners',
       'Italian', 'Tapas/Small Plates', 'Caterers', '

In [27]:
df.drop(['Unnamed: 0', 'Unnamed: 0_x', 'Unnamed: 0.1'], axis = 1, inplace = True)

In [29]:
df.to_csv("sentiment_numbered_reviews.csv")

In [31]:
df.columns

Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text',
       'useful', 'user_id', 'address',
       ...
       'RestaurantsTableService', 'RestaurantsTakeOut', 'Smoking',
       'WheelchairAccessible', 'WiFi', 'Unnamed: 0_y', 'neg', 'neu', 'pos',
       'compound'],
      dtype='object', length=237)

In [37]:
reviews_head = df.head()

In [41]:
dummies = pd.get_dummies(reviews_head,
                         prefix = {'state': 'St',
                                   'neighborhood':'nbrhd',
                                   'city': 'city',
                                   'postal_code': 'pc'},
                         columns = ['state', 'neighborhood', 'city', 'postal_code'])

In [49]:
dummies.columns

Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text',
       'useful', 'user_id', 'address', 'attributes', 'categories', 'hours',
       'is_open', 'latitude', 'longitude', 'name', 'review_count',
       'Review_Num', 'Hot Dogs', 'Salad', 'French', 'Active Life', 'Barbeque',
       'Middle Eastern', 'Korean', 'Tapas Bars', 'British', 'Indian',
       'Poutineries', 'Restaurants', 'Coffee & Tea', 'Creperies', 'Falafel',
       'Sandwiches', 'Arts & Entertainment', 'Poke', 'Noodles',
       'Do-It-Yourself Food', 'Bakeries', 'Ethiopian', 'Asian Fusion',
       'Hawaiian', 'Japanese', 'Local Flavor', 'Vegan', 'Taiwanese', 'Tex-Mex',
       'Food Court', 'Dance Clubs', 'Cajun/Creole', 'Sushi Bars', 'Desserts',
       'Pan Asian', 'Food Trucks', 'Turkish', 'Beer', 'Venues & Event Spaces',
       'Breakfast & Brunch', 'Bars', 'Pakistani', 'Salvadoran', 'Diners',
       'Italian', 'Tapas/Small Plates', 'Caterers', 'Wine Bars', 'Waffles',
       'Chinese', 'Sports Bars',

## Final DF?

In [50]:
reviews = pd.read_csv("Complete_reviews.csv")

/hpc/applications/anaconda/3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15,166,181,188,196,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,220,221,222,223,224,225) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [56]:
reviews.columns

Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text',
       'useful', 'user_id', 'is_open', 'latitude', 'longitude', 'name',
       'postal_code', 'review_count', 'Review_Num', 'Hot Dogs', 'Salad',
       'French', 'Active Life', 'Barbeque', 'Middle Eastern', 'Korean',
       'Tapas Bars', 'British', 'Indian', 'Poutineries', 'Restaurants',
       'Coffee & Tea', 'Creperies', 'Falafel', 'Sandwiches',
       'Arts & Entertainment', 'Poke', 'Noodles', 'Do-It-Yourself Food',
       'Bakeries', 'Ethiopian', 'Asian Fusion', 'Hawaiian', 'Japanese',
       'Local Flavor', 'Vegan', 'Taiwanese', 'Tex-Mex', 'Food Court',
       'Dance Clubs', 'Cajun/Creole', 'Sushi Bars', 'Desserts', 'Pan Asian',
       'Food Trucks', 'Turkish', 'Beer', 'Venues & Event Spaces',
       'Breakfast & Brunch', 'Bars', 'Pakistani', 'Salvadoran', 'Diners',
       'Italian', 'Tapas/Small Plates', 'Caterers', 'Wine Bars', 'Waffles',
       'Chinese', 'Sports Bars', 'Mexican', 'Hot Pot', 'Delicates

In [57]:
reviews.drop('Unnamed: 0_y', axis = 1, inplace = True)

In [58]:
reviews.to_csv("Complete_reviews.csv")

## Checking DF

In [2]:
reviews = pd.read_csv("Complete_reviews_head_barebones.csv")

In [3]:
reviews.shape

(100, 588)

In [5]:
reviews.drop(["business_id", "date", "user_id", "latitude", "longitude", "name", "postal_code"],
             axis = 1, 
             inplace = True)

In [10]:
reviews.to_csv("Complete_reviews_head_barebones.csv")

In [ ]:
reviews.drop(["Unnamed: 0", 'Unnamed: 0.1', 'Unnamed: 0.1.1'],
             axis = 1, 
             inplace = True)

In [4]:
reviews.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'business_id', 'cool',
       'date', 'funny', 'review_id', 'stars', 'text',
       ...
       'Reservations_True', 'Wheelchair_False', 'Wheelchair_True', 'wifi_free',
       'wifi_no', 'wifi_paid', 'BikePark_False', 'BikePark_True',
       'CreditCards_False', 'CreditCards_True'],
      dtype='object', length=588)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [29]:
X = reviews['text']
y = reviews['stars']

In [25]:
vectorizer = TfidfVectorizer(stop_words = "english",
                            max_df = 0.7,
                            min_df = 5,
                            token_pattern = '[A-Za-z][A-Za-z]+')
tfidf = vectorizer.fit_transform(X)
print(len(vectorizer.get_feature_names()))
print(vectorizer.get_feature_names()[:20])


298
['absolutely', 'actually', 'aged', 'amazing', 'ambiance', 'appetizer', 'arrived', 'asked', 'asparagus', 'ate', 'atmosphere', 'attentive', 'authentic', 'average', 'away', 'awesome', 'baby', 'bacon', 'bad', 'baked']


In [47]:
np.concatenate((reviews.columns, np.array(vectorizer.get_feature_names())))

array(['cool', 'funny', 'review_id', 'stars', 'text', 'useful', 'is_open',
       'review_count', 'Review_Num', 'Hot Dogs', 'Salad', 'French',
       'Active Life', 'Barbeque', 'Middle Eastern', 'Korean', 'Tapas Bars',
       'British', 'Indian', 'Poutineries', 'Restaurants', 'Coffee & Tea',
       'Creperies', 'Falafel', 'Sandwiches', 'Arts & Entertainment',
       'Poke', 'Noodles', 'Do-It-Yourself Food', 'Bakeries', 'Ethiopian',
       'Asian Fusion', 'Hawaiian', 'Japanese', 'Local Flavor', 'Vegan',
       'Taiwanese', 'Tex-Mex', 'Food Court', 'Dance Clubs', 'Cajun/Creole',
       'Sushi Bars', 'Desserts', 'Pan Asian', 'Food Trucks', 'Turkish',
       'Beer', 'Venues & Event Spaces', 'Breakfast & Brunch', 'Bars',
       'Pakistani', 'Salvadoran', 'Diners', 'Italian',
       'Tapas/Small Plates', 'Caterers', 'Wine Bars', 'Waffles', 'Chinese',
       'Sports Bars', 'Mexican', 'Hot Pot', 'Delicatessen', 'Steakhouses',
       'Wraps', 'Lebanese', 'Karaoke', 'Filipino', 'Gastropubs',
   

In [28]:
type(tfidf)

scipy.sparse.csr.csr_matrix

In [52]:
reviews.groupby("Review_Num").std()

,cool,funny,stars,useful,is_open,review_count,Hot Dogs,Salad,French,Active Life,...,city_Mesa,city_Mississauga,city_Montréal,city_Phoenix,city_Pittsburgh,city_Scottsdale,city_Toronto,alcohol_beer_and_wine,alcohol_full_bar,alcohol_none
Review_Num,,,,,,,,,,,,,,,,,,,,,
1,0.707107,0.000000,2.828427,3.535534,0.0,1076.216521,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.707107,0.707107,0.0
2,1.414214,0.000000,2.121320,4.242641,0.0,1076.216521,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.707107,0.707107,0.0
3,0.000000,0.707107,1.414214,0.707107,0.0,1076.216521,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.707107,0.707107,0.0
4,1.414214,0.000000,0.000000,0.000000,0.0,1076.216521,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.707107,0.707107,0.0
5,0.707107,0.707107,0.000000,0.707107,0.0,1076.216521,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.707107,0.707107,0.0
6,0.000000,0.000000,0.000000,0.000000,0.0,1076.216521,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.707107,0.707107,0.0
7,0.707107,0.000000,0.000000,1.414214,0.0,1076.216521,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.707107,0.707107,0.0
8,0.707107,0.000000,0.707107,1.414214,0.0,1076.216521,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.707107,0.707107,0.0
9,2.121320,2.828427,0.000000,2.828427,0.0,1076.216521,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.707107,0.707107,0.0


In [5]:
reviews.dtypes[reviews.dtypes == np.object].index

Index(['business_id', 'date', 'review_id', 'text', 'user_id', 'name',
       'postal_code'],
      dtype='object')

## Checking Final DF

In [ ]:
reviews = pd.read_csv("/users/aschams/scratch/Complete_reviews.csv")
y = reviews['stars'].copy()
reviews_text = reviews[['text','review_id']].copy()
reviews.groupby("Review_Num").mean().to_csv("/users/aschams/scratch/Average_by_review_number.csv")
reviews.groupby("Review_Num").std().to_csv("/users/aschams/scratch/stdev_by_review_number.csv")
reviews.drop(['Unnamed: 0', 'Unnamed: 0.1','review_id', 'business_id', "date", 'text', 'stars', 'user_id', 'name', 'Unnamed: 0_y', 'postal_code'],
            axis = 1,
            inplace = True)








/hpc/applications/anaconda/3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15,193,201,202,203,204,205,206,207,208,209,211,212,213) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data = pd.read_csv("/users/aschams/scratch/Complete_reviews_barebones_noTFIDF.csv")